# linegraph - ipyrun tutorial 

In [1]:
import sys
import pathlib

sys.path.append(str(pathlib.Path('../../src').resolve())) # append ipyrun. TODO: for dev only. remove at build time.
sys.path.append(str(pathlib.Path('../../../ipyautoui/src').resolve())) # append ipyautoui. TODO: for dev only. remove at build time.

import subprocess
from pydantic import validator

from ipyrun import RunApp, BatchApp, ConfigShell, DefaultConfigShell, RunShellActions 
from ipyrun.runshell import DisplayfileDefinition, create_displayfile_renderer, ConfigBatch, DefaultConfigShell, DefaultBatchActions, BatchShellActions

from ipyautoui import AutoUi, AutoUiConfig
from ipyautoui import DisplayFiles
from ipyautoui.displayfile import display_python_file

In [2]:
#  for supporting info see: 
#  ?RunApp
#  ?DefaultConfigShell
#  ?RunShellActions

In [3]:
#?RunApp
#?DefaultConfigShell

## Define Brief for App

- given a set of inputs, create a line graph of a defined range
- __note__. `ipyrun` will deal with the caching of the data





## Create a working directory 

In [4]:
from seedir import seedir

DIR_CORE = pathlib.Path('./linegraph_core').resolve()
FPTH_SCRIPT = DIR_CORE / 'script_linegraph.py'
FPTH_INPUTS_SCHEMA = DIR_CORE / 'input_schema_linegraph.py'
DIR_APPDATA = pathlib.Path('./linegraph_appdata').resolve()

seedir(exclude_folders=['.ipynb_checkpoints', '__pycache__'], style='emoji')

📁 linegraph/
├─📄 in-test.lg.json
├─📄 linegraph_app.ipynb
├─📁 linegraph_appdata/
│ ├─📁 00-linegraph/
│ │ ├─📄 config-shell_handler.json
│ │ ├─📄 in-00-linegraph.lg.json
│ │ ├─📄 out-00-linegraph.csv
│ │ └─📄 out-00-linegraph.plotly.json
│ ├─📁 01-linegraph/
│ │ ├─📄 config-shell_handler.json
│ │ └─📄 in-01-linegraph.lg.json
│ └─📄 config-shell_handler.json
├─📄 linegraph_app_tutorial.ipynb
├─📁 linegraph_core/
│ ├─📄 input_schema_linegraph.py
│ ├─📄 script_linegraph.py
│ └─📄 __init__.py
├─📄 out-test.csv
├─📄 out-test.plotly.json
└─📄 __init__.py


## App Inputs

### Create pydantic model schema for App inputs

- what are the required input fields required for plotting a line graph? 
- create a pydantic schema of these inputs

In [5]:
from linegraph_core.input_schema_linegraph import LineGraph
display_python_file(FPTH_INPUTS_SCHEMA)  # it is advised that input_schema's get there own python file


```Python
from pydantic import BaseModel , Field, conint
import typing
import pathlib

class LineGraph(BaseModel):
    """parameters to define a simple `y=m*x + c` line graph"""
    title: str = Field(default='line equation', description='add chart title here')
    m: float = Field(default=2, description='gradient')
    c: float = Field(default=5, description='intercept')
    x_range: tuple[int, int] = Field(default=(0,5), ge=0, le=50, description='x-range for chart')
    y_range: tuple[int, int] = Field(default=(0,5), ge=0, le=50, description='y-range for chart')
        
if __name__ == "__main__":
    lg = LineGraph()
```


### Create an inputs UI from model with AutoUi

- check the UI looks correct based on the pydantic model
- save the test file (`test.lg.json`) so we can test loading it back in


In [6]:

# ?AutoUiConfig
config_lgui =AutoUiConfig(ext='.lg.json', pydantic_model=LineGraph)
lg = LineGraph()
AutoUi(pydantic_obj=lg, path='in-test.lg.json', config_autoui=config_lgui)

AutoUi(children=(VBox(children=(HBox(children=(SaveButtonBar(children=(ToggleButton(value=False, disabled=True…

### Create a DisplayFiles renderer, and check it works

- extend the default DisplayFiles renderer to be able to render `*.lg.json` files


In [9]:

# ?DisplayfileDefinition
lg_inputs_dfd = DisplayfileDefinition(path=FPTH_INPUTS_SCHEMA,
                      obj_name='LineGraph',
                      ftype='in',
                      ext='.lg.json'
)
lg_inputs_renderer = create_displayfile_renderer(lg_inputs_dfd)

DisplayFiles(paths=['in-test.lg.json'],user_file_renderers=lg_inputs_renderer)

## Write main App code

- This is where you put the main code that fulfils the original brief
- We are using the `RunShell` App so the main code lives in a script that is executed on the command line
- There must be a way to pass input variables to this script
    - The default way in `RunShell` is that the filepath of the inputs and outputs files are passed as a `sys.argv`s
- In the simple example below, the script can be tested by simply running:
    - `python -O linegraph_core/script_linegraph.py in-test.lg.json out-test.csv out-test.plotly.json`

In [11]:
subprocess.call('python -O linegraph_core/script_linegraph.py in-test.lg.json out-test.csv out-test.plotly.json', shell=True)

fpth_in = in-test.lg.json
fpth_out_csv = out-test.csv
fpth_out_plotly = out-test.plotly.json


0

In [10]:
display_python_file(FPTH_SCRIPT)


```Python
"""a test script for ipyrun that plots a line graph based on inputs

Returns:
    fpth_csv (str): csv file with plot data
    fpth_plotly (str): plotly json of plot
"""
import pandas as pd
import numpy as np
import plotly.express as px
import json
from input_schema_linegraph import LineGraph

def fpth_chg_extension(fpth, new_ext="plotly"):
    return os.path.splitext(fpth)[0] + "." + new_ext

def graph(
    formula, x_range, title="", display_plot=False, fpth_save_plot=None, returndf=True
):
    """
    generic graph plotter.
    pass a  and a range to plot over
    and returns a dataframe.

    Args:
        formula(str): formula as a string
        x_range(np.arange): formula as a string
        **plot(booll:
        **returndf(bool)
    Returns:
        df(pd.DataFrame):
    """
    try:
        x = np.array(x_range)
        y = eval(formula)
    except:
        li = []
        y = [li.append(eval(formula)) for x in x_range]
        x = x_range
        y = np.array(li)
    if len(title) > 0:
        title = title + "<br>" + "y = " + formula
    else:
        title = "y = " + formula
    try:
        df = pd.DataFrame({"x": x, "y": y}, columns=["x", "y"])
    except:
        df = {"x": x, "y": y}
    if display_plot:
        fig = px.line(df, x="x", y="y", title=title, template="plotly_white")
        fig.show()
    if fpth_save_plot is not None:
        fig = px.line(df, x="x", y="y", title=title, template="plotly_white")
        data = fig.write_json(fpth_save_plot)
    if returndf:
        return df

def main(fpth_in, fpth_out_csv, fpth_out_plotly):
    """
    Args:
        fpth_in (str)
        fpth_out_csv (str)
        fpth_out_plotly (str)
    Returns:
        fpth_out_csv
    """
    inputs = LineGraph.parse_file(fpth_in)
    formula = "{} * x + {}".format(inputs.m, inputs.c)
    x_range = np.arange(inputs.x_range[0], inputs.x_range[1], step=1)
    df = graph(
        formula,
        x_range,
        fpth_save_plot=fpth_out_plotly,
        returndf=True,
        title=inputs.title,
    )
    df.to_csv(fpth_out_csv)
    return df


if __name__ == "__main__":
    if __debug__:
        import os
        import pathlib
        os.chdir(pathlib.Path(__file__).parent)
        fpth_in = "inputs-line_graph.lg.json"
        fpth_out_csv = "line_graph-output.csv"
        fpth_out_plotly = "line_graph-output.plotly.json"
        main(fpth_in, fpth_out_csv, fpth_out_plotly)
    else:
        import sys
        fpth_in = sys.argv[1]
        fpth_out_csv = sys.argv[2]
        fpth_out_plotly = sys.argv[3]
        print(f"fpth_in = {fpth_in}")
        print(f"fpth_out_csv = {fpth_out_csv}")
        print(f"fpth_out_plotly = {fpth_out_plotly}")
        main(fpth_in, fpth_out_csv, fpth_out_plotly)

```

## Create a RunApp

### Set-up App Config

- Now we have an inputs file, and a script that can be executed to create the desired outputs
- We must now configure the App which will handle the caching of input and output data
    - This allows the Notebook to be run as an App, such that a single Notebook can be run by any job, with the job data in the job 

In [24]:
#?DefaultConfigShell

In [16]:
lg_inputs_dfd.dict()

{'path': PosixPath('/mnt/c/engDev/git_mf/ipyrun/examples/linegraph/linegraph_core/input_schema_linegraph.py'),
 'obj_name': 'LineGraph',
 'module_name': 'input_schema_linegraph',
 'ftype': <FiletypeEnum.input: 'in'>,
 'ext': '.lg.json'}

In [13]:
# DefaultConfigShell defines how to cache data to be run by the App
# ?DefaultConfigShell  # show help

config = DefaultConfigShell(
    fpth_script=FPTH_SCRIPT,
    fdir_appdata=DIR_APPDATA,
    displayfile_definitions=[lg_inputs_dfd],
)
display(config.dict())

{'index': 0,
 'fpth_script': PosixPath('/mnt/c/engDev/git_mf/ipyrun/examples/linegraph/linegraph_core/script_linegraph.py'),
 'name': 'linegraph',
 'long_name': '00 - Linegraph',
 'key': '00-linegraph',
 'fdir_appdata': PosixPath('/mnt/c/engDev/git_mf/ipyrun/examples/linegraph/linegraph_appdata/00-linegraph'),
 'in_batch': False,
 'status': None,
 'update_config_at_runtime': False,
 'displayfile_definitions': [{'path': PosixPath('/mnt/c/engDev/git_mf/ipyrun/examples/linegraph/linegraph_core/input_schema_linegraph.py'),
   'obj_name': 'LineGraph',
   'module_name': 'input_schema_linegraph',
   'ftype': <FiletypeEnum.input: 'in'>,
   'ext': '.lg.json'}],
 'displayfile_inputs_kwargs': {},
 'displayfile_outputs_kwargs': {},
 'fpths_inputs': [PosixPath('/mnt/c/engDev/git_mf/ipyrun/examples/linegraph/linegraph_appdata/00-linegraph/in-00-linegraph.lg.json')],
 'fpths_outputs': [],
 'fpth_params': None,
 'fpth_config': PosixPath('/mnt/c/engDev/git_mf/ipyrun/examples/linegraph/linegraph_appdata

- As you can see above, given this set of inputs the shell command has been generated 
- We can either: 
    - 1. explictly give fpths_outputs as a variable, or 
    - 2. extend `DefaultConfigShell` to generate `fpths_outputs` from other variables
- option 2. is favourable as it can then be easily extended to an arbitrary number of runs, and the names of the output files will follow the same logical naming structure as the input files

In [20]:
class LineGraphConfigShell(DefaultConfigShell):

    @validator("fpths_outputs", always=True)
    def _fpths_outputs(cls, v, values):
        fdir = values["fdir_appdata"]
        key = values["key"]
        paths = [
            fdir / ("out-" + key + ".csv"),
            fdir / ("out-" + key + ".plotly.json"),
        ]
        return paths
    
config = LineGraphConfigShell(
    fpth_script= DIR_CORE/'script_linegraph.py', 
    fdir_appdata=DIR_APPDATA,
    displayfile_definitions=[lg_inputs_dfd],
)
#display(config.dict())

### Build the RunApp

In [19]:
#  RunApp is described as UI first -
#  as such it can be loaded and viewed before all of the buttons have been assigned useful callable functions 
#  pressing the buttons below will yield simple default outputs

RunApp(config=config, cls_actions=RunShellActions)

RunApp(children=(HBox(children=(Checkbox(value=False, indent=False, layout=Layout(height='40px', max_width='20…

## Create a BatchApp

- BatchApp's support an arbitrary number of runs 
- This extends the app to be able to add and remove runs to suit the users requirements
- To be able to do this, we need to be able to build a new RunApp on a button click
    - this requires `LineGraphConfigShell` to be fully defined apart from a single variable (the `index` that will be passed from the BatchApp) 

In [21]:

class LineGraphConfigShell(DefaultConfigShell):
    @validator("fpth_script", always=True, pre=True)
    def _set_fpth_script(cls, v, values):
        return FPTH_SCRIPT

    @validator("fpths_outputs", always=True)
    def _fpths_outputs(cls, v, values):
        fdir = values["fdir_appdata"]
        key = values["key"]
        paths = [
            fdir / ("out-" + key + ".csv"),
            fdir / ("out-" + key + ".plotly.json"),
        ]
        return paths

    @validator("displayfile_definitions", always=True)
    def _displayfile_definitions(cls, v, values):
        return [
            DisplayfileDefinition(
                path=FPTH_INPUTS_SCHEMA,
                obj_name="LineGraph",
                ext=".lg.json",
                ftype='in',
            )
        ]

    @validator("displayfile_inputs_kwargs", always=True)
    def _displayfile_inputs_kwargs(cls, v, values):
        return dict(patterns="*")

    @validator("displayfile_outputs_kwargs", always=True)
    def _displayfile_outputs_kwargs(cls, v, values):
        return dict(patterns="*.plotly.json")

config = LineGraphConfigShell(fdir_appdata=DIR_APPDATA)
run_app = RunApp(config, cls_actions=RunShellActions)  # cls_ui=RunUi,
display(run_app)

RunApp(children=(HBox(children=(Checkbox(value=False, indent=False, layout=Layout(height='40px', max_width='20…

In [22]:

config_batch = ConfigBatch(
    fdir_root=DIR_APPDATA,
    cls_config=LineGraphConfigShell,
    title="""# Plot Straight Lines\n### example RunApp""",
)
from devtools import debug

debug(config_batch)
print('---')

/tmp/ipykernel_1490/3736187265.py:8 <module>
    config_batch: ConfigBatch(
        fdir_root=PosixPath('/mnt/c/engDev/git_mf/ipyrun/examples/linegraph/linegraph_appdata'),
        fpth_config=PosixPath('/mnt/c/engDev/git_mf/ipyrun/examples/linegraph/linegraph_appdata/config-shell_handler.json'),
        title=(
            '# Plot Straight Lines\n'
            '### example RunApp'
        ),
        status=None,
        cls_actions=<class 'ipyrun.runshell.RunShellActions'>,
        cls_app=functools.partial(<class 'ipyrun.runui.RunApp'>, cls_actions=<class 'ipyrun.runshell.RunShellActions'>),
        cls_config=<class '__main__.LineGraphConfigShell'>,
        configs=[],
    ) (ConfigBatch)
---


### Extend Batch Actions

In [23]:


class LineGraphConfigBatch(ConfigBatch):


    @validator("cls_config", always=True)
    def _cls_config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        return LineGraphConfigShell

class LineGraphBatchActions(BatchShellActions):
    @validator("config", always=True)
    def _config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        if type(v) == dict:
            v = LineGraphConfigBatch(**v)
        return v

    @validator("runlog_show", always=True)
    def _runlog_show(cls, v, values):
        return None

config_batch = LineGraphConfigBatch(
    fdir_root=DIR_APPDATA,
    # cls_config=MyConfigShell,
    title="""# Plot Straight Lines\n### example RunApp""",
)
if config_batch.fpth_config.is_file():
    config_batch = LineGraphConfigBatch.parse_file(config_batch.fpth_config)
app = BatchApp(config_batch, cls_actions=LineGraphBatchActions)
display(app)

BatchApp(children=(VBox(children=(HTML(value='<h1>Plot Straight Lines</h1>\n<h3>example RunApp</h3>'), HBox(ch…